# Prevendo 3 meses a frente
Por fim iremos finalizar a previsão dos clientes que nao deram churn neste mês mas que poderiam dar no proximo ou em M2, M3 com os valores otimos da busca bayesiana. 

Criaremos também uma função de visualização para os resultados do modelo.

In [ ]:
join_tables['safra'] = pd.to_datetime(join_tables.safra, format='%Y%m', errors='coerce').dt.strftime('%Y%m')

In [ ]:
# Notebook 4
join_tables['churn_1'] = join_tables.groupby('msno')['is_churn'].shift(-1)
join_tables['churn_2'] = join_tables.groupby('msno')['is_churn'].shift(-2)
join_tables['churn_3'] = join_tables.groupby('msno')['is_churn'].shift(-3)

In [ ]:
# Previsão para M1
x1m = join_tables.dropna(subset=['churn_1']).filter(feature_importance_df)
y_1m = join_tables.dropna(subset=['churn_1'])['churn_1']

# Previsão para M2
x2m = join_tables.dropna(subset=['churn_2']).filter(feature_importance_df)
y_2m = join_tables.dropna(subset=['churn_2'])['churn_2']

# Previsão para M3
x3m = join_tables.dropna(subset=['churn_3']).filter(feature_importance_df)
y_3m = join_tables.dropna(subset=['churn_3'])['churn_3']


In [ ]:
join_tables.sort_values(by=['msno', 'safra'], ascending = False, inplace = True)

In [ ]:
# Treinamento e teste para previsão de 1 mês (M1)
X_train1, X_test1m, y_train_1m, y_test_1m = train_test_split(x1m, y_1m, test_size=0.3, random_state=42)
costumer_churn_prediction(xgc, X_train1, X_test1m, y_train_1m, y_test_1m, "features", threshold_plot=True)

# Treinamento e teste para previsão de 2 meses (M2)
X_train2, X_test2m, y_train_2m, y_test_2m = train_test_split(x2m, y_2m, test_size=0.3, random_state=42)
costumer_churn_prediction(xgc, X_train2, X_test2m, y_train_2m, y_test_2m, "features", threshold_plot=True)

# Treinamento e teste para previsão de 3 meses (M3)
X_train3, X_test3m, y_train_3m, y_test_3m = train_test_split(x3m, y_3m, test_size=0.3, random_state=42)
costumer_churn_prediction(xgc, X_train3, X_test3m, y_train_3m, y_test_3m, "features", threshold_plot=True)


In [ ]:
# Previsão e cálculo do churn em cada janela
for period, y_pred, y_test in zip(['1 mês', '2 meses', '3 meses'], 
                                  [y_pred_1m, y_pred_2m, y_pred_3m], 
                                  [y_test_1m, y_test_2m, y_test_3m]):
    total_churn = (y_pred == 1).sum()
    print(f'Percentual de clientes com churn em {period}: {total_churn / len(y_pred):.2%}')


In [ ]:
#Importing libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from math import * # module math
import matplotlib.pyplot as plt # visualization
from PIL import Image
import seaborn as sns # visualization
import itertools
import io
import plotly.offline as py # visualization
py.init_notebook_mode(connected=True) # visualization
import plotly.graph_objs as go # visualization
from plotly.subplots import make_subplots
import plotly.figure_factory as ff # visualization
import warnings
warnings.filterwarnings("ignore")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import statsmodels.api as sm
from yellowbrick.classifier import DiscriminationThreshold

%matplotlib inline

In [ ]:
def costumer_churn_prediction(algorithm, training_x, testing_x, training_y, testing_y, cf, threshold_plot):
    #model
    algorithm.fit(training_x, training_y)
    predictions = algorithm.predict(testing_x)
    probabilities = algorithm.predict_proba(testing_x)
        
    print('Algorithm:', type(algorithm).__name__)
    print("\nClassification report:\n", classification_report(testing_y, predictions))
    print("Accuracy Score:", accuracy_score(testing_y, predictions))
    
    #confusion matrix
    conf_matrix = confusion_matrix(testing_y, predictions)
    #roc_auc_score
    model_roc_auc = roc_auc_score(testing_y, predictions) 
    print("Area under curve:", model_roc_auc,"\n")
    
    fpr, tpr, thresholds = roc_curve(testing_y, probabilities[:,1])
     
    #plot confusion matrix
    trace1 = go.Heatmap(z = conf_matrix,
                        x = ["Not churn", "Churn"],
                        y = ["Not churn", "Churn"],
                        showscale = False, colorscale = "Picnic",
                        name = "Confusion matrix")
    
    #plot roc curve
    trace2 = go.Scatter(x = fpr, y = tpr,
                        name = "Roc: " + str(model_roc_auc),
                        line = dict(color = ('rgb(22, 96, 167)'), width = 2))
    trace3 = go.Scatter(x = [0,1], y = [0,1],
                        line = dict(color = ('rgb(205, 12, 24)'), width = 2,
                        dash = 'dot'))
    
    if cf in ['coefficients', 'features']:
        if cf == 'coefficients':
            coefficients = pd.DataFrame(algorithm.coef_.ravel())
        elif cf == 'features':
            coefficients = pd.DataFrame(algorithm.feature_importances_)
        
        column_df = pd.DataFrame(training_x.columns.tolist())
        coef_sumry = (pd.merge(coefficients, column_df, left_index=True, 
                               right_index=True, how="left"))
        coef_sumry.columns = ["coefficients", "features"]
        coef_sumry = coef_sumry.sort_values(by = "coefficients", ascending=False)
        
        #plot coeffs
        trace4 = go.Bar(x = coef_sumry["features"], y = coef_sumry["coefficients"], 
                        name = "coefficients",
                        marker = dict(color = coef_sumry["coefficients"],
                                      colorscale = "Picnic",
                                      line = dict(width = .6, color = "black")
                                     )
                       )
        #subplots
        fig = make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
                                subplot_titles=('Confusion matrix',
                                                'Receiver operating characteristic',
                                                'Feature importances')
                           )  
        fig.append_trace(trace1,1,1)
        fig.append_trace(trace2,1,2)
        fig.append_trace(trace3,1,2)
        fig.append_trace(trace4,2,1)
        fig['layout'].update(showlegend=False, title="Model performance",
                             autosize=False, height = 900, width = 800,
                             plot_bgcolor = 'rgba(240,240,240, 0.95)',
                             paper_bgcolor = 'rgba(240,240,240, 0.95)',
                             margin = dict(b = 195))
        fig["layout"]["xaxis2"].update(dict(title = "false positive rate"))
        fig["layout"]["yaxis2"].update(dict(title = "true positive rate"))
        fig["layout"]["xaxis3"].update(dict(showgrid = True, tickfont = dict(size = 10), tickangle = 90))
        
    elif cf == 'None':
        #subplots
        fig = make_subplots(rows=1, cols=2,
                            subplot_titles=('Confusion matrix',
                                            'Receiver operating characteristic')
                           )
        fig.append_trace(trace1,1,1)
        fig.append_trace(trace2,1,2)
        fig.append_trace(trace3,1,2)
        fig['layout'].update(showlegend=False, title="Model performance",
                         autosize=False, height = 500, width = 800,
                         plot_bgcolor = 'rgba(240,240,240,0.95)',
                         paper_bgcolor = 'rgba(240,240,240,0.95)',
                         margin = dict(b = 195))
        fig["layout"]["xaxis2"].update(dict(title = "false positive rate"))
        fig["layout"]["yaxis2"].update(dict(title = "true positive rate"))  
        
    py.iplot(fig)
    
    if threshold_plot == True: 
        visualizer = DiscriminationThreshold(algorithm)
        visualizer.fit(training_x,training_y)
        visualizer.poof()

In [ ]:
# Separar variáveis explicativas e alvo para previsão de churn no próximo mês

# Dividir os dados em conjuntos de treinamento e teste
X_train1, X_test1m, y_train_1m, y_test_1m = train_test_split(x1m, y_1m, test_size=0.3, random_state=42)


from xgboost import XGBClassifier

xgc = XGBClassifier(scale_pos_weight=scale_pos_weight,  # Ajuste para classes desbalanceadas
   use_label_encoder=False, reg_lambda =  0.1, reg_alpha = 0.644, n_estimators = 50,
                         max_depth = 2, learning_rate = 0.11)

costumer_churn_prediction(xgc, X_train1, X_test1m, y_train_1m, y_test_1m, "features", threshold_plot=True)

In [ ]:
print('Percentual de clientes com churn:',(total_churn_1m)/(len(y_pred_1m)))

## **Conclusão:**
Conseguimos identificar 6% clientes com possível  churn nos proximos três meses.
Entendendo que X% ficam ativos pós ação, conseguiremos reter X%.

É importante passar a lista destes clientes para que a area de negocio entre em contato tentando fortalecer e restabelecer o relacionamento com o objetivo de reduzir o provável churn.

Para finalizar, iremos realizar a analise não supervisionada, agrupar nossos clientes e tentar buscar padrões de churn e comportamento.